In [1]:
from jinja2 import Environment, FileSystemLoader
import json

In [2]:
def page_link(a):
    if ' ' in a:
        return a.replace(" ", "_")
    else: 
        return a

In [3]:
f = open('jsons/general.json')
general = json.load(f)

e = open('jsons/pi.json')
prin = json.load(e)

g = open('jsons/post_docs.json')
post_docs = json.load(g)

h = open('jsons/grad_students.json')
grad_students = json.load(h)

i = open('jsons/undergrads.json')
undergrads = json.load(i)

In [4]:
environment = Environment(loader=FileSystemLoader("templates/"))
environment.globals['page_link'] = page_link
template = environment.get_template("individual_person.html.j2")

In [5]:
for role in [prin, post_docs, grad_students, undergrads]:
    for person in role:
        filename = f"{ page_link(person['name']) }.html"
        content = template.render(
            general=general,
            person=person
        )
        with open(filename, mode="w", encoding="utf-8") as page:
            page.write(content)


In [6]:
f.close()
e.close()
g.close()
h.close()
i.close()